In [1]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm

In [7]:
class PolicyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        out = self.model(x)
        return out
    
    def get_action(self, x):
        out = self(x)
        action = torch.distributions.Categorical(out).sample()
        return action

class ValueModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
        )

    def forward(self, x):
        out = self.model(x)
        return out

class Agent:
    def __init__(self):
        self.policy_model = PolicyModel()
        self.value_model = ValueModel()
        self.policy_optimizer = optim.Adam(list(self.policy_model.parameters()), lr=0.01)
        self.value_optimizer = optim.Adam(list(self.value_model.parameters()), lr=0.01)
        self.gamma = 0.99
    
    def update(self, policy_loss, value_loss):
        self.policy_optimizer.zero_grad()
        policy_loss.backward(retain_graph=True)
        self.policy_optimizer.step()

        self.value_optimizer.zero_grad()
        value_loss.backward()
        self.value_optimizer.step()
    
    def run_episode(self, env_name="CartPole-v1"):
        # records state, action, reward for each step
        env = gym.make(env_name)  # Remove render_mode for training
        episode_obs = torch.tensor([])
        episode_aux = torch.tensor([])
        observation, info = env.reset()
        obs_output = torch.tensor(observation, dtype=torch.float32)[None, :]
        terminated = False
        truncated = False
        while not terminated and not truncated:
            obs_input = obs_output
            action = self.policy_model.get_action(obs_input)
            observation, reward, terminated, truncated, info = env.step(action.numpy()[0])
            obs_output = torch.tensor(observation, dtype=torch.float32)[None, :]
            episode_obs = torch.cat([episode_obs, torch.cat((obs_input, obs_output))[None, :]])
            episode_aux = torch.cat([episode_aux, torch.tensor([action, reward])[None, :]])
        return episode_obs, episode_aux

    def get_losses(self, all_episodes):
        policy_loss = torch.tensor([])
        value_loss = torch.tensor([])
        for episode_obs, episode_aux in all_episodes:
            advantages = episode_aux[:, 1] + self.gamma * self.value_model(episode_obs[:, 1])[:,0] - self.value_model(episode_obs[:, 0])[:,0]
            outs = self.policy_model(episode_obs[:, 0])
            log_probs = torch.log(outs[torch.arange(len(episode_obs)), episode_aux[:, 0].to(torch.int64)])
            policy_loss = torch.cat([policy_loss, -advantages * log_probs])
            value_loss = torch.cat([value_loss, advantages**2])
        return policy_loss.mean(), value_loss.mean()

    def avg_reward(self, episodes):
        return torch.tensor([episode[1][:, 1].sum() for episode in episodes]).mean()

    def train(self, num_episodes=100, print_loss=True):
        # collects episodes, updates policy and value models
        all_episodes = []
        for i in range(num_episodes):
            episode = self.run_episode()
            all_episodes.append(episode)
        
        policy_loss, value_loss = self.get_losses(all_episodes)
        total_reward = self.avg_reward(all_episodes).item()

        self.update(policy_loss, value_loss)
        if print_loss:
            print(f"Episode {i} policy loss: {policy_loss.item()}")
            print(f"Episode {i} value loss: {value_loss.item()}")
            print(f"Episode {i} average total reward: {total_reward}")
        return (policy_loss, value_loss, total_reward)
    
    def demo(self, env_name="CartPole-v1"):
        env = gym.make(env_name, render_mode="human")
        observation, info = env.reset()
        obs_output = torch.tensor(observation, dtype=torch.float32)[None, :]
        terminated = False
        truncated = False
        while not terminated and not truncated:
            action = self.policy_model.get_action(obs_output)
            observation, reward, terminated, truncated, info = env.step(action.numpy()[0])
            obs_output = torch.tensor(observation, dtype=torch.float32)[None, :]
        env.close()

In [6]:
agent = Agent()
policy_losses = []
value_losses = []
total_rewards = []
for i in tqdm(range(1000), desc="Training"):
    policy_loss, value_loss, total_reward = agent.train(num_episodes=100, print_loss=False)
    policy_losses.append(policy_loss)
    value_losses.append(value_loss)
    total_rewards.append(total_reward)
    if total_reward > 500:
        print(f"Episode {i} average total reward: {total_reward}")
        break

agent.demo()

Training:  23%|██▎       | 228/1000 [04:02<1:05:42,  5.11s/it]